## **1. Load Machine Learning Model**

In [21]:
import os
from machinelearning import FraudModel

path = os.getcwd()
path = path + "\\machinelearning\\"
path

'c:\\Users\\anita\\OneDrive\\Desktop\\stream processing\\machinelearning\\'

## **2. Load Data Old Information From PostgreSQL** 

In [22]:
import json
import pandas as pd
import urllib.parse
from sqlalchemy import create_engine

# Baca konfigurasi dari file JSON
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Ambil konfigurasi PostgreSQL dari JSON
postgres_config = config['postgres']
username = postgres_config['username']
password = urllib.parse.quote_plus(postgres_config['password'])
host = postgres_config['host']
port = postgres_config['port']
database = postgres_config['database']

# URL koneksi ke PostgreSQL
db_url = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(db_url)

df = pd.read_sql_query('SELECT * FROM old_information_devina;', engine)
df.head(5)

,nameOrig,oldbalanceOrg,nameDest,oldbalanceDest
0,C1231006815,170136.0,M1979787155,0.0
1,C1666544295,21249.0,M2044282225,0.0
2,C1305486145,181.0,C553264065,0.0
3,C840083671,181.0,C38997010,21182.0
4,C2048537720,41554.0,M1230701703,0.0


## **3. Build Connection MongoDB**

In [23]:
import json
from pymongo import MongoClient

# Baca konfigurasi dari file JSON
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Ambil konfigurasi mongo dari JSON
mongo_config = config['mongo']
mongo_username = mongo_config['username']
mongo_password = urllib.parse.quote_plus(mongo_config['password'])
mongo_host = mongo_config['host']
mongo_port = mongo_config['port']

try:
    server = MongoClient(f"mongodb://{mongo_username}:{mongo_password}@{mongo_host}:{mongo_port}/")
    db = server.admin
    server_status = db.command("ping")
    print("MongoDB connection successful:", server_status)

    databases = server.list_database_names()
    print("Databases:", databases)

except Exception as e:
    print("An error occurred:", e)

MongoDB connection successful: {'ok': 1.0}
Databases: ['admin', 'config', 'ftde02', 'ftde3', 'local']


## **4. Load Data New Information From Kafka Producer**

In [ ]:
1. Ambil Data dari Kafka Producer
2. Merge / Join dengan Data dari PostgreSQL
3. Prediksi Transaksi
4. Insert kedalam MongoDB

In [ ]:
import json
from kafka import KafkaConsumer

if __name__ == "__main__":

    # Baca konfigurasi dari config.json
    with open('config.json', 'r') as config_file:
        config = json.load(config_file)

    # Ambil konfigurasi Kafka
    kafka_config = config["kafka"]
    kafka_host = kafka_config["broker_host"]
    kafka_topic = kafka_config["topic"]

    # Ambil konfigurasi Mongo
    mongo_config = config['mongo']
    mongo_database = mongo_config['database']

    consumer = KafkaConsumer(kafka_topic, bootstrap_servers=kafka_host)
    print("Starting the consumer")
    
    for msg in consumer:
        print(f"Records = {json.loads(msg.value)}")
        
        # Parsing pesan Kafka
        data = json.loads(msg.value)
        produder = pd.DataFrame([data])
        
        #join old information di postgreSQL
        data = produder.merge(df, how='inner', on=['nameOrig','nameDest'])

        data_predict = data.drop(['nameOrig','nameDest'], axis=1)
        data_predict = data_predict.to_dict('index')[0]

        result = FraudModel.runModel(data_predict, path)
        data['predict'] = result
        
        #insert mongodb
        db = server[mongo_database]
        collection = db["fraud_devina"]

        if isinstance(data.to_dict('index')[0], list):
            collection.insert_many(data.to_dict('index')[0])
        else:
            collection.insert_one(data.to_dict('index')[0])        
            print("Data telah disimpan ke MongoDB")

Starting the consumer
